In [ ]:
import numpy as np
from numpy import ndarray

In [ ]:
import plotly.graph_objects as go
def save_figure_to_file(fig, path, file_name):
    # Comment this line if you get an error
    fig.write_image(path + '/' + file_name + '.svg')

In [ ]:
# Adjust this parameter to change the running time
# Vary between 100 and 1,000. Default = 800. Fast = 10
n_epochs = 800

In [ ]:
x = np.array([-1, -1e-3, 1e-3, 1])

fig = go.Figure()
fig.add_trace(go.Scatter(x=x, y= np.heaviside(x,0.5), name='heaviside'))
fig.update_layout(width=600, yaxis_range=[-0.1, 1.1], title='heaviside function')
fig.show()

In [ ]:
save_figure_to_file(fig, '../Slides/ANN', 'fig1')

In [ ]:
x = np.hstack( (np.linspace(-4,-1e-3,32), np.linspace(1e-3,4,32)) )

fig = go.Figure()
fig.add_trace(go.Scatter(x=x, y= np.heaviside(x,0.5), name='heaviside'))
fig.add_trace(go.Scatter(x=x, y= 1/(1 + np.exp(-x)), name='sigmoid'))
fig.add_trace(go.Scatter(x=x, y= np.tanh(x), name='tanh'))
fig.add_trace(go.Scatter(x=x, y= np.maximum(x, 0), name='relu'))
fig.update_layout(width=600, yaxis_range=[-1.1, 2.1])
fig.show()

In [ ]:
save_figure_to_file(fig, '../Slides/ANN', 'fig2')

In [ ]:
x = np.hstack( (np.linspace(-4,-1e-3,32), np.linspace(1e-3,4,32)) )

fig = go.Figure()
fig.add_trace(go.Scatter(x=x, y= 1/(1 + np.exp(-x)), name='sigmoid', line_width=8))
fig.update_layout(width=450, yaxis_range=[-0.5, 1.5])
fig.show()

In [ ]:
def fun(x):
    return 1. / (1. + x**2)

X_train = np.linspace(-1,1,1025)
y_train = fun(X_train)

X_valid = np.linspace(-1,1,257)
y_valid = fun(X_valid)

X_test = np.linspace(-1,1,129)
y_test = fun(X_test)

In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=X_train, y=y_train))
fig.update_layout(width=600,title='Function to approximate', xaxis_title='x')
fig.show()

In [ ]:
save_figure_to_file(fig, '../Slides/TF_Keras', 'fig1')

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import optimizers

learning_rate = 0.1

In [ ]:
tf.random.set_seed(2020)
model = keras.models.Sequential()
model.add(keras.layers.InputLayer(input_shape=1))
model.add(keras.layers.Dense(4, activation="tanh"))
model.add(keras.layers.Dense(4, activation="tanh"))
model.add(keras.layers.Dense(4, activation="tanh"))
model.add(keras.layers.Dense(1, activation="linear"))

model.summary()

In [ ]:
keras.utils.plot_model(model)

In [ ]:
model.layers

In [ ]:
hidden1 = model.layers[1]
print(hidden1.name)

In [ ]:
weights, biases = hidden1.get_weights()
print(weights.shape)
print(biases.shape)

In [ ]:
sgd = optimizers.SGD(lr=learning_rate)
model.compile(loss='mse', optimizer=sgd, metrics=['mse','mae'])
history = model.fit(X_train, y_train, epochs=n_epochs,
                    validation_data=(X_valid, y_valid))

In [ ]:
def plot_history(fig, history):
    error = history.history["mse"]
    error_val = history.history["val_mse"]
    fig.add_trace(go.Scatter(y=error,name='error training'))
    fig.add_trace(go.Scatter(y=error_val, name='error validation'))
    fig.update_layout(width=600, yaxis_type="log", 
                      yaxis_range=[-7,-3],
                      xaxis_title='epoch')    

fig = go.Figure()
plot_history(fig, history)
fig.show()

In [ ]:
save_figure_to_file(fig, '../Slides/TF_Keras', 'fig2')

In [ ]:
model.evaluate(X_test, y_test)

In [ ]:
y_pred = np.array( model.predict(X_test) ).flatten()
fig = go.Figure()
fig.add_trace(go.Scatter(x=X_test, y=y_test, name='exact'))
fig.add_trace(go.Scatter(x=X_test, y=y_pred, name='model'))
fig.update_layout(width=600, xaxis_title='x')
fig.show()

In [ ]:
save_figure_to_file(fig, '../Slides/TF_Keras', 'fig3')

In [ ]:
def plot_error(fig, model, X_test, y_test):
    y_pred = np.array( model.predict(X_test) ).flatten()
    error = abs(y_test-y_pred)
    fig.add_trace(go.Scatter(x = X_test, y = error))
    fig.update_layout(width=600, yaxis_type='log', title='absolute error vs x',
                     xaxis_range=[-1,1], yaxis_range=[-6,-1], xaxis_title='x')
    return error

fig = go.Figure()
error_seq = plot_error(fig, model, X_test, y_test)
fig.show()

In [ ]:
save_figure_to_file(fig, '../Slides/TF_Keras', 'fig4')

In [ ]:
from tensorflow.keras import layers, Model

In [ ]:
tf.random.set_seed(2020)
input_ = layers.Input(shape=1)
hidden1 = layers.Dense(4, activation="tanh")(input_)
hidden2 = layers.Dense(4, activation="tanh")(hidden1)
hidden3 = layers.Dense(4, activation="tanh")(hidden2)
concat = layers.Concatenate()([hidden1, hidden2, hidden3])
output = layers.Dense(1, activation="linear")(concat)
model = Model(inputs=[input_], outputs=[output])

model.summary()

In [ ]:
keras.utils.plot_model(model)

In [ ]:
model.compile(loss='mse',
              optimizer=sgd,
              metrics=['mse','mae'])
history = model.fit(X_train, y_train, epochs=n_epochs,
                    validation_data=(X_valid, y_valid))

In [ ]:
fig = go.Figure()
plot_history(fig, history)
fig.show()

In [ ]:
save_figure_to_file(fig, '../Slides/TF_Keras', 'fig5')

In [ ]:
fig = go.Figure()
error_cat = plot_error(fig, model, X_test, y_test)
fig.show()

In [ ]:
save_figure_to_file(fig, '../Slides/TF_Keras', 'fig6')

In [ ]:
tf.random.set_seed(2020)

class MyModel(keras.Model):
    def __init__(self, **kwargs):
        super().__init__(**kwargs) # handles standard args (e.g., name)
        self.hidden1 = keras.layers.Dense(4, activation="relu")
        self.hidden2 = keras.layers.Dense(4, activation="relu")
        self.hidden3 = keras.layers.Dense(4, activation="relu")        
        self.out = keras.layers.Dense(1, activation="linear")

    def call(self, input_):
        hidden1 = self.hidden1(input_)
        hidden2 = self.hidden2(hidden1)
        hidden3 = self.hidden3(hidden2)        
        concat = layers.Concatenate()([input_, hidden3])
        return self.out(concat)

model = MyModel()

In [ ]:
model.compile(loss='mse',
              optimizer=sgd,
              metrics=['mse','mae'])
history = model.fit(X_train, y_train, epochs=n_epochs,
                    validation_data=(X_valid, y_valid))

In [ ]:
model.summary()

In [ ]:
fig = go.Figure()
plot_history(fig, history)
fig.show()

In [ ]:
save_figure_to_file(fig, '../Slides/TF_Keras', 'fig7')

In [ ]:
fig = go.Figure()
error_relu = plot_error(fig, model, X_test, y_test)
fig.show()

In [ ]:
save_figure_to_file(fig, '../Slides/TF_Keras', 'fig8')

In [ ]:
def multi_input(X):
    return np.transpose( np.vstack( (2 * X**2 - 1, 8 * X**4 - 8 * X**2 + 1 )))

In [ ]:
tf.random.set_seed(2020)

import math

X_train = np.linspace(0,1,513)
y_train = fun(X_train)

X_valid = np.linspace(0,1,129)
y_valid = fun(X_valid)

X_test = np.linspace(0,1,65)
y_test = fun(X_test)

input_ = layers.Input(shape=2)
hidden1 = layers.Dense(4, activation="tanh")(input_)
hidden2 = layers.Dense(4, activation="tanh")(hidden1)
hidden3 = layers.Dense(4, activation="tanh")(hidden2)
concat = layers.Concatenate()([input_, hidden3])
output = layers.Dense(1, activation="linear")(hidden3)
model = Model(inputs=[input_], outputs=[output])

model.summary()

In [ ]:
model.compile(loss='mse',
              optimizer=sgd,
              metrics=['mse','mae'])

X_train_3 = multi_input(X_train)
X_valid_3 = multi_input(X_valid)

history = model.fit(X_train_3, y_train, epochs=n_epochs,
                    validation_data=(X_valid_3, y_valid))

In [ ]:
fig = go.Figure()
plot_history(fig, history)
fig.show()

In [ ]:
save_figure_to_file(fig, '../Slides/TF_Keras', 'fig9')

In [ ]:
fig = go.Figure()
y_pred = np.array( model.predict( multi_input(X_test) ) ).flatten()
error_mX = abs(y_test-y_pred)
fig.add_trace(go.Scatter(x = X_test, y = error_mX))
fig.update_layout(width=600, yaxis_type="log")
fig.show()

In [ ]:
save_figure_to_file(fig, '../Slides/TF_Keras', 'fig10')

In [ ]:
error_multiX = np.concatenate( (error_mX[::-1], error_mX[1:]) )

In [ ]:
fig = go.Figure()
X_test = np.linspace(-1,1,129)
fig.add_trace(go.Scatter(x = X_test, y = error_relu, name = 'relu DNN'))
fig.add_trace(go.Scatter(x = X_test, y = error_cat, name = 'concat DNN'))
fig.add_trace(go.Scatter(x = X_test, y = error_seq, name = 'seq DNN'))
fig.add_trace(go.Scatter(x = X_test, y = error_multiX, name = 'multi X'))
fig.update_layout(width=900, yaxis_type="log", xaxis_title="x")
fig.show()

In [ ]:
save_figure_to_file(fig, '../Slides/TF_Keras', 'fig11')